In [1]:
# https://github.com/Leoloh/tutorials/blob/master/python_tutorial.ipynb

In [2]:
from catboost.datasets import titanic
import numpy as np

train_df, test_df = titanic()

In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
null_value_stats = train_df.isnull().sum(axis = 0)

null_value_stats[null_value_stats != 0]

Age         177
Cabin       687
Embarked      2
dtype: int64

In [7]:
train_df.fillna(-999, inplace = True)
test_df.fillna(-999, inplace = True)

In [8]:
X = train_df.drop('Survived', axis = 1)
y = train_df.Survived

In [9]:
print(X.dtypes)

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size = 0.75, random_state = 42)

X_test = test_df

C:\Users\LUY1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [11]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

In [12]:
model = CatBoostClassifier(custom_loss = ['Accuracy'],
                          random_seed = 42,
                          logging_level = 'Silent')

In [14]:
categorical_feature_indices = np.where(X.dtypes != np.float)[0]

print(categorical_feature_indices)

[ 0  1  2  3  5  6  7  9 10]


In [15]:
X

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,-999,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,-999,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,-999,S
5,6,3,"Moran, Mr. James",male,-999.0,0,0,330877,8.4583,-999,Q
6,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,-999,S
8,9,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,-999,S
9,10,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,-999,C


In [16]:
# ignore the titanic dataset

In [17]:
import pandas as pd

In [19]:
df = pd.read_pickle('df_mod_3.pkl')

df.head()

,term,grade,home_ownership,verification_status,purpose,application_type,loan_amnt,int_rate,annual_inc,dti,Emp_Length_Years,Earliest_Cr_Line_Yr,Default
400246,0,1,2,2,1,0,5225.0,11.99,17627.0,29.84,6.0,1994,1
223922,0,1,3,1,1,0,10000.0,13.11,40400.0,18.42,2.0,2004,1
361504,0,4,3,1,2,0,6000.0,22.15,72000.0,15.83,4.0,1993,1
261157,0,4,3,0,2,0,5000.0,18.54,28000.0,16.80,0.0,1959,1
808762,1,5,3,2,2,0,19475.0,23.99,55000.0,34.98,9.0,1989,1


In [20]:
df.columns.values

array(['term', 'grade', 'home_ownership', 'verification_status',
       'purpose', 'application_type', 'loan_amnt', 'int_rate',
       'annual_inc', 'dti', 'Emp_Length_Years', 'Earliest_Cr_Line_Yr',
       'Default'], dtype=object)

In [21]:
X = df[['term', 'grade', 'home_ownership', 'verification_status',
       'purpose', 'application_type', 'loan_amnt', 'int_rate',
       'annual_inc', 'dti', 'Emp_Length_Years', 'Earliest_Cr_Line_Yr']]

y = df['Default']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, random_state = 123)

C:\Users\LUY1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [23]:
X_train.head()

,term,grade,home_ownership,verification_status,purpose,application_type,loan_amnt,int_rate,annual_inc,dti,Emp_Length_Years,Earliest_Cr_Line_Yr
457551,0,0,0,2,2,0,24000.0,8.90,71000.0,21.50,10.0,1986
602171,1,1,0,0,1,0,28000.0,9.17,135000.0,17.15,3.0,2001
454782,0,2,3,1,2,0,24000.0,14.47,110000.0,25.90,10.0,1988
284103,0,4,3,0,2,0,4950.0,22.15,38000.0,19.84,10.0,2004
626811,0,1,3,0,2,0,20000.0,8.18,72000.0,24.44,10.0,1997


In [26]:
# Model Training

In [24]:
mod = CatBoostClassifier(custom_loss = ['Accuracy'],
                        random_seed = 42,
                        logging_level = 'Silent')

In [25]:
mod.fit(X_train, y_train,
       cat_features = [0,1,2,3,4,5],
       eval_set = (X_test, y_test),
       plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [27]:
# Model Cross-validattion

In [28]:
cv_params = mod.get_params()

print(cv_params)

{'custom_loss': ['Accuracy'], 'logging_level': 'Silent', 'random_seed': 42, 'loss_function': 'Logloss'}


In [29]:
cv_data = cv(Pool(X_train, y_train, cat_features = [0,1,2,3,4,5]),
            cv_params,
            plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [30]:
print('Best validation accuracy score: {:.2f}+/-{:.2f} on step {}'.format(np.max(cv_data['test-Accuracy-mean']),
                                                                         cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
                                                                         np.argmax(cv_data['test-Accuracy-mean'])))

Best validation accuracy score: 0.70+/-0.02 on step 45


C:\Users\LUY1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:52: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


In [31]:
#Model Applying

In [32]:
predictions = mod.predict(X_test)
predictions_probs = mod.predict_proba(X_test)


In [33]:
print(predictions[:10])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [34]:
print(predictions_probs[:10])

[[0.46900041 0.53099959]
 [0.92827899 0.07172101]
 [0.56182793 0.43817207]
 [0.60737327 0.39262673]
 [0.85615028 0.14384972]
 [0.60791088 0.39208912]
 [0.88344772 0.11655228]
 [0.67236313 0.32763687]
 [0.77908456 0.22091544]
 [0.72275034 0.27724966]]


In [35]:
# CatBoost Features

In [37]:
mod_without_seed = CatBoostClassifier(iterations = 10,
                                     logging_level = 'Silent')

mod_without_seed.fit(X_train, y_train,
                    cat_features = [0,1,2,3,4,5])

In [38]:
print('Random seed assigned for this model: {}'.format(mod_without_seed.random_seed_))

Random seed assigned for this model: 0


In [42]:
params = {'iterations': 500,
         'learning_rate': 0.1,
         'eval_metric': 'Accuracy',
         'random_seed': 42,
         'logging_level': 'Silent',
         'use_best_model': False}

In [43]:
train_pool = Pool(X_train, y_train, cat_features = [0,1,2,3,4,5])
validation_pool = Pool(X_test, y_test, cat_features = [0,1,2,3,4,5])

In [44]:
mod2 = CatBoostClassifier(**params)

mod2.fit(train_pool,
        eval_set = validation_pool)

In [45]:
best_model_params = params.copy()
best_model_params

{'iterations': 500,
 'learning_rate': 0.1,
 'eval_metric': 'Accuracy',
 'random_seed': 42,
 'logging_level': 'Silent',
 'use_best_model': False}

In [46]:
best_model_params.update({'use_best_model': True})

best_model_params

{'iterations': 500,
 'learning_rate': 0.1,
 'eval_metric': 'Accuracy',
 'random_seed': 42,
 'logging_level': 'Silent',
 'use_best_model': True}

In [47]:
best_mod = CatBoostClassifier(**best_model_params)

best_mod.fit(train_pool,
            eval_set = validation_pool)

In [49]:
print('Simple model validation accuracy: {:.4}'.format(accuracy_score(y_test, mod2.predict(X_test))))

Simple model validation accuracy: 0.6933


In [50]:
print('Best model validation accuracy: {:.4}'.format(accuracy_score(y_test, best_mod.predict(X_test))))

Best model validation accuracy: 0.6987


In [51]:
# Early Stopping

In [52]:
%%time

mod3 = CatBoostClassifier(**params)

mod3.fit(train_pool,
        eval_set = validation_pool)

Wall time: 15.2 s


In [59]:
earlystop_params = params.copy()
earlystop_params

{'iterations': 500,
 'learning_rate': 0.1,
 'eval_metric': 'Accuracy',
 'random_seed': 42,
 'logging_level': 'Silent',
 'use_best_model': False}

In [60]:
earlystop_params.update({'od_type': 'Iter',
                        'od_wait': 40})

earlystop_params

{'iterations': 500,
 'learning_rate': 0.1,
 'eval_metric': 'Accuracy',
 'random_seed': 42,
 'logging_level': 'Silent',
 'use_best_model': False,
 'od_type': 'Iter',
 'od_wait': 40}

In [61]:
%%time

earlystop_mod = CatBoostClassifier(**earlystop_params)

earlystop_mod.fit(train_pool,
                 eval_set = validation_pool)

Wall time: 1.48 s


In [63]:
print('Simple model tree count: {}'.format(mod3.tree_count_))

Simple model tree count: 500


In [64]:
print('Simple model validation accuracy: {:.4}'.format(accuracy_score(y_test, mod3.predict(X_test))))

Simple model validation accuracy: 0.6933


In [66]:
print('Early-stopped model tree count: {}'.format(earlystop_mod.tree_count_))

Early-stopped model tree count: 67


In [67]:
print('Early-stopped model validation accuracy: {:.4}'.format(accuracy_score(y_test, earlystop_mod.predict(X_test))))

Early-stopped model validation accuracy: 0.6853


In [68]:
# Using Baseline

current_params = params.copy()
current_params

{'iterations': 500,
 'learning_rate': 0.1,
 'eval_metric': 'Accuracy',
 'random_seed': 42,
 'logging_level': 'Silent',
 'use_best_model': False}

In [69]:
current_params.update({'iterations': 10})

current_params

{'iterations': 10,
 'learning_rate': 0.1,
 'eval_metric': 'Accuracy',
 'random_seed': 42,
 'logging_level': 'Silent',
 'use_best_model': False}

In [70]:
mod4 = CatBoostClassifier(**current_params)

In [71]:
mod4.fit(X_train, y_train,
        cat_features = [0,1,2,3,4,5])

In [73]:
baseline = mod4.predict(X_train,
                       prediction_type = 'RawFormulaVal')

In [74]:
baseline

array([-1.1752293 , -1.0790896 , -0.33697361, ..., -0.0928922 ,
       -0.97796631, -0.99949464])

In [75]:
mod4.fit(X_train, y_train,
        cat_features = [0,1,2,3,4,5],
        baseline = baseline)

In [76]:
#Snapshot Support

params_with_snapshot = params.copy()
params_with_snapshot

{'iterations': 500,
 'learning_rate': 0.1,
 'eval_metric': 'Accuracy',
 'random_seed': 42,
 'logging_level': 'Silent',
 'use_best_model': False}

In [77]:
params_with_snapshot.update({'iterations': 5,
                            'learning_rate': 0.5,
                            'logging_level': 'Verbose'})

params_with_snapshot

{'iterations': 5,
 'learning_rate': 0.5,
 'eval_metric': 'Accuracy',
 'random_seed': 42,
 'logging_level': 'Verbose',
 'use_best_model': False}

In [78]:
mod5 = CatBoostClassifier(**params_with_snapshot)

In [79]:
mod5.fit(train_pool,
        eval_set = validation_pool,
        save_snapshot = True)

0:	learn: 0.7048889	test: 0.6760000	best: 0.6760000 (0)	total: 58ms	remaining: 232ms
1:	learn: 0.7048889	test: 0.6760000	best: 0.6760000 (0)	total: 83.9ms	remaining: 126ms
2:	learn: 0.7048889	test: 0.6786667	best: 0.6786667 (2)	total: 128ms	remaining: 85.6ms
3:	learn: 0.7111111	test: 0.6866667	best: 0.6866667 (3)	total: 170ms	remaining: 42.5ms
4:	learn: 0.7142222	test: 0.6866667	best: 0.6866667 (3)	total: 209ms	remaining: 0us

bestTest = 0.6866666667
bestIteration = 3



In [81]:
params_with_snapshot.update({'iterations': 10,
                            'learning_rate': 0.1})

params_with_snapshot

{'iterations': 10,
 'learning_rate': 0.1,
 'eval_metric': 'Accuracy',
 'random_seed': 42,
 'logging_level': 'Verbose',
 'use_best_model': False}

In [82]:
mod6 = CatBoostClassifier(**params_with_snapshot)

In [83]:
mod6.fit(train_pool,
        eval_set = validation_pool,
        save_snapshot = True)

5:	learn: 0.7146667	test: 0.6853333	best: 0.6866667 (3)	total: 231ms	remaining: 90.5ms
6:	learn: 0.7151111	test: 0.6853333	best: 0.6866667 (3)	total: 261ms	remaining: 77.8ms
7:	learn: 0.7142222	test: 0.6840000	best: 0.6866667 (3)	total: 304ms	remaining: 63.5ms
8:	learn: 0.7142222	test: 0.6826667	best: 0.6866667 (3)	total: 335ms	remaining: 31.7ms
9:	learn: 0.7146667	test: 0.6826667	best: 0.6866667 (3)	total: 364ms	remaining: 0us

bestTest = 0.6866666667
bestIteration = 3



In [87]:
# User Defined Objective Function

class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)
        
        result = []
        for index in range(len(targets)):
            e = np.exp(approxes[index])
            p = e/(1+e)
            der1 = (1-p) if targets[index] > 0.0 else -p
            der2 = -p*(1-p)
            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]
            result.append((der1, der2))
        return result
        

In [88]:
mod7 = CatBoostClassifier(iterations = 10,
                         random_seed = 42,
                         loss_function = LoglossObjective(),
                         eval_metric = 'Logloss')



In [89]:
mod7.fit(train_pool)

0:	learn: 0.6866484	total: 130ms	remaining: 1.17s
1:	learn: 0.6812768	total: 223ms	remaining: 891ms
2:	learn: 0.6758821	total: 316ms	remaining: 738ms
3:	learn: 0.6714932	total: 388ms	remaining: 583ms
4:	learn: 0.6667222	total: 483ms	remaining: 483ms
5:	learn: 0.6625542	total: 590ms	remaining: 393ms
6:	learn: 0.6579559	total: 683ms	remaining: 293ms
7:	learn: 0.6533777	total: 766ms	remaining: 191ms
8:	learn: 0.6496519	total: 851ms	remaining: 94.6ms
9:	learn: 0.6455980	total: 938ms	remaining: 0us


In [91]:
preds_raw = mod7.predict(X_test,
                        prediction_type = 'RawFormulaVal')

preds_raw

array([-1.36194666e-02, -3.60360744e-01, -8.48650150e-02, -1.02688665e-01,
       -3.46855224e-01, -1.91605799e-01, -3.42647079e-01, -1.05294622e-01,
       -2.91172275e-01, -1.63309461e-01, -3.27261274e-01, -1.36194666e-02,
        6.45963404e-02, -1.04030367e-01, -2.84678446e-02, -9.86053002e-02,
       -3.60360744e-01, -1.06984864e-01, -7.96313668e-02, -3.46855224e-01,
       -3.60360744e-01, -1.17535104e-01, -1.92922878e-01, -3.54774015e-01,
       -9.43587285e-02, -2.87324812e-02, -1.02688665e-01, -3.29322131e-01,
       -3.29322131e-01, -1.01916717e-01,  7.51465807e-02, -1.64998991e-01,
       -1.08929574e-01, -4.89861616e-02, -9.43587285e-02, -5.87269335e-02,
       -3.53016372e-01, -1.23362433e-02, -3.56834872e-01, -1.62867381e-01,
       -3.49490500e-01, -2.95974500e-02, -3.46855224e-01, -2.84678446e-02,
       -3.21977759e-01, -2.93564723e-01, -3.29896550e-01, -3.56472305e-01,
       -3.90765040e-02, -3.60360744e-01, -1.66845309e-01, -3.49490500e-01,
       -3.49991451e-01, -

In [92]:
# https://github.com/catboost/catboost/issues/172

In [93]:
X_train.head()

,term,grade,home_ownership,verification_status,purpose,application_type,loan_amnt,int_rate,annual_inc,dti,Emp_Length_Years,Earliest_Cr_Line_Yr
457551,0,0,0,2,2,0,24000.0,8.90,71000.0,21.50,10.0,1986
602171,1,1,0,0,1,0,28000.0,9.17,135000.0,17.15,3.0,2001
454782,0,2,3,1,2,0,24000.0,14.47,110000.0,25.90,10.0,1988
284103,0,4,3,0,2,0,4950.0,22.15,38000.0,19.84,10.0,2004
626811,0,1,3,0,2,0,20000.0,8.18,72000.0,24.44,10.0,1997


In [94]:
cat_features = [0,1,2,3,4,5]

grid_params = {'depth': [4,6,8],
              'learning_rate': [0.01, 0.03, 0.1]}

In [100]:
mod_cb = CatBoostClassifier(iterations = 500,
                           learning_rate = 0.03)

In [101]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator = mod_cb,
                   param_grid = grid_params,
                   scoring = 'roc_auc',
                   cv = 5,
                   verbose = 1)

In [102]:
grid.fit(X_train, y_train,
        cat_features = cat_features)

0:	learn: 0.6909752	total: 24.7ms	remaining: 12.3s
1:	learn: 0.6890759	total: 50ms	remaining: 12.4s
2:	learn: 0.6866114	total: 80.1ms	remaining: 13.3s
3:	learn: 0.6839364	total: 112ms	remaining: 13.9s
4:	learn: 0.6815336	total: 142ms	remaining: 14.1s
5:	learn: 0.6794725	total: 173ms	remaining: 14.2s
6:	learn: 0.6775805	total: 194ms	remaining: 13.6s
7:	learn: 0.6756674	total: 207ms	remaining: 12.7s
8:	learn: 0.6734996	total: 227ms	remaining: 12.4s
9:	learn: 0.6716869	total: 249ms	remaining: 12.2s
10:	learn: 0.6697877	total: 266ms	remaining: 11.8s
11:	learn: 0.6678250	total: 293ms	remaining: 11.9s
12:	learn: 0.6664034	total: 304ms	remaining: 11.4s
13:	learn: 0.6642109	total: 321ms	remaining: 11.1s
14:	learn: 0.6621673	total: 338ms	remaining: 10.9s
15:	learn: 0.6602475	total: 354ms	remaining: 10.7s
16:	learn: 0.6582434	total: 370ms	remaining: 10.5s
17:	learn: 0.6565979	total: 387ms	remaining: 10.4s
18:	learn: 0.6550378	total: 412ms	remaining: 10.4s
19:	learn: 0.6532563	total: 438ms	remain

168:	learn: 0.5680793	total: 3.32s	remaining: 6.51s
169:	learn: 0.5678938	total: 3.34s	remaining: 6.49s
170:	learn: 0.5678322	total: 3.36s	remaining: 6.46s
171:	learn: 0.5677195	total: 3.37s	remaining: 6.43s
172:	learn: 0.5675619	total: 3.39s	remaining: 6.41s
173:	learn: 0.5673032	total: 3.41s	remaining: 6.38s
174:	learn: 0.5672534	total: 3.42s	remaining: 6.34s
175:	learn: 0.5670667	total: 3.43s	remaining: 6.32s
176:	learn: 0.5669159	total: 3.44s	remaining: 6.28s
177:	learn: 0.5668952	total: 3.45s	remaining: 6.24s
178:	learn: 0.5667506	total: 3.46s	remaining: 6.21s
179:	learn: 0.5665930	total: 3.48s	remaining: 6.19s
180:	learn: 0.5664610	total: 3.5s	remaining: 6.17s
181:	learn: 0.5662648	total: 3.52s	remaining: 6.15s
182:	learn: 0.5660707	total: 3.54s	remaining: 6.14s
183:	learn: 0.5659957	total: 3.56s	remaining: 6.11s
184:	learn: 0.5657902	total: 3.57s	remaining: 6.08s
185:	learn: 0.5655836	total: 3.59s	remaining: 6.06s
186:	learn: 0.5654256	total: 3.6s	remaining: 6.03s
187:	learn: 0.

327:	learn: 0.5529029	total: 5.87s	remaining: 3.08s
328:	learn: 0.5527994	total: 5.89s	remaining: 3.06s
329:	learn: 0.5527627	total: 5.92s	remaining: 3.05s
330:	learn: 0.5526181	total: 5.93s	remaining: 3.03s
331:	learn: 0.5525562	total: 5.95s	remaining: 3.01s
332:	learn: 0.5525365	total: 5.96s	remaining: 2.99s
333:	learn: 0.5524448	total: 5.98s	remaining: 2.97s
334:	learn: 0.5523444	total: 5.99s	remaining: 2.95s
335:	learn: 0.5522018	total: 6.01s	remaining: 2.93s
336:	learn: 0.5521781	total: 6.02s	remaining: 2.91s
337:	learn: 0.5521104	total: 6.04s	remaining: 2.89s
338:	learn: 0.5520379	total: 6.05s	remaining: 2.88s
339:	learn: 0.5519653	total: 6.07s	remaining: 2.86s
340:	learn: 0.5519310	total: 6.09s	remaining: 2.84s
341:	learn: 0.5519129	total: 6.11s	remaining: 2.82s
342:	learn: 0.5518634	total: 6.12s	remaining: 2.8s
343:	learn: 0.5517930	total: 6.14s	remaining: 2.78s
344:	learn: 0.5517716	total: 6.14s	remaining: 2.76s
345:	learn: 0.5517145	total: 6.16s	remaining: 2.74s
346:	learn: 0

492:	learn: 0.5453211	total: 8.39s	remaining: 119ms
493:	learn: 0.5452157	total: 8.41s	remaining: 102ms
494:	learn: 0.5450974	total: 8.43s	remaining: 85.2ms
495:	learn: 0.5450727	total: 8.46s	remaining: 68.2ms
496:	learn: 0.5450686	total: 8.47s	remaining: 51.1ms
497:	learn: 0.5450427	total: 8.48s	remaining: 34.1ms
498:	learn: 0.5450178	total: 8.5s	remaining: 17ms
499:	learn: 0.5449244	total: 8.51s	remaining: 0us
0:	learn: 0.6906653	total: 20.4ms	remaining: 10.2s
1:	learn: 0.6883331	total: 45.5ms	remaining: 11.3s
2:	learn: 0.6858574	total: 58.4ms	remaining: 9.67s
3:	learn: 0.6838535	total: 74.9ms	remaining: 9.28s
4:	learn: 0.6816341	total: 91.1ms	remaining: 9.02s
5:	learn: 0.6795505	total: 107ms	remaining: 8.78s
6:	learn: 0.6770349	total: 122ms	remaining: 8.61s
7:	learn: 0.6748784	total: 138ms	remaining: 8.5s
8:	learn: 0.6728694	total: 157ms	remaining: 8.54s
9:	learn: 0.6711050	total: 174ms	remaining: 8.52s
10:	learn: 0.6691040	total: 191ms	remaining: 8.47s
11:	learn: 0.6673413	total: 2

156:	learn: 0.5695263	total: 2.57s	remaining: 5.62s
157:	learn: 0.5693109	total: 2.59s	remaining: 5.61s
158:	learn: 0.5691863	total: 2.61s	remaining: 5.59s
159:	learn: 0.5691078	total: 2.62s	remaining: 5.56s
160:	learn: 0.5689851	total: 2.64s	remaining: 5.55s
161:	learn: 0.5687779	total: 2.65s	remaining: 5.54s
162:	learn: 0.5685885	total: 2.67s	remaining: 5.52s
163:	learn: 0.5683257	total: 2.68s	remaining: 5.5s
164:	learn: 0.5681744	total: 2.7s	remaining: 5.48s
165:	learn: 0.5681454	total: 2.71s	remaining: 5.45s
166:	learn: 0.5678159	total: 2.73s	remaining: 5.44s
167:	learn: 0.5676385	total: 2.74s	remaining: 5.42s
168:	learn: 0.5674087	total: 2.76s	remaining: 5.41s
169:	learn: 0.5671566	total: 2.78s	remaining: 5.4s
170:	learn: 0.5670518	total: 2.8s	remaining: 5.38s
171:	learn: 0.5669553	total: 2.81s	remaining: 5.35s
172:	learn: 0.5668255	total: 2.83s	remaining: 5.35s
173:	learn: 0.5668012	total: 2.84s	remaining: 5.33s
174:	learn: 0.5666485	total: 2.86s	remaining: 5.31s
175:	learn: 0.56

...

480:	learn: 0.3008071	total: 34.9s	remaining: 1.38s
481:	learn: 0.3003945	total: 35s	remaining: 1.3s
482:	learn: 0.2993211	total: 35s	remaining: 1.23s
483:	learn: 0.2992461	total: 35.1s	remaining: 1.16s
484:	learn: 0.2990269	total: 35.2s	remaining: 1.09s
485:	learn: 0.2988731	total: 35.2s	remaining: 1.01s
486:	learn: 0.2988523	total: 35.3s	remaining: 942ms
487:	learn: 0.2983131	total: 35.4s	remaining: 870ms
488:	learn: 0.2981859	total: 35.5s	remaining: 798ms
489:	learn: 0.2981835	total: 35.5s	remaining: 725ms
490:	learn: 0.2981454	total: 35.6s	remaining: 653ms
491:	learn: 0.2980189	total: 35.7s	remaining: 581ms
492:	learn: 0.2978037	total: 35.9s	remaining: 509ms
493:	learn: 0.2977885	total: 36s	remaining: 438ms
494:	learn: 0.2976488	total: 36.2s	remaining: 365ms
495:	learn: 0.2975506	total: 36.3s	remaining: 293ms
496:	learn: 0.2974665	total: 36.5s	remaining: 221ms
497:	learn: 0.2971246	total: 36.7s	remaining: 147ms
498:	learn: 0.2963570	total: 36.9s	remaining: 73.9ms
499:	learn: 0.2962

146:	learn: 0.5461597	total: 4.53s	remaining: 10.9s
147:	learn: 0.5461596	total: 4.55s	remaining: 10.8s
148:	learn: 0.5457495	total: 4.59s	remaining: 10.8s
149:	learn: 0.5457459	total: 4.61s	remaining: 10.7s
150:	learn: 0.5457361	total: 4.62s	remaining: 10.7s
151:	learn: 0.5456283	total: 4.65s	remaining: 10.7s
152:	learn: 0.5455762	total: 4.68s	remaining: 10.6s
153:	learn: 0.5453169	total: 4.72s	remaining: 10.6s
154:	learn: 0.5449152	total: 4.74s	remaining: 10.6s
155:	learn: 0.5449135	total: 4.76s	remaining: 10.5s
156:	learn: 0.5449110	total: 4.77s	remaining: 10.4s
157:	learn: 0.5449110	total: 4.78s	remaining: 10.3s
158:	learn: 0.5447111	total: 4.81s	remaining: 10.3s
159:	learn: 0.5444995	total: 4.83s	remaining: 10.3s
160:	learn: 0.5444662	total: 4.84s	remaining: 10.2s
161:	learn: 0.5444316	total: 4.85s	remaining: 10.1s
162:	learn: 0.5444008	total: 4.86s	remaining: 10s
163:	learn: 0.5444008	total: 4.87s	remaining: 9.97s
164:	learn: 0.5443880	total: 4.88s	remaining: 9.91s
165:	learn: 0.

311:	learn: 0.5250822	total: 8.59s	remaining: 5.17s
312:	learn: 0.5250516	total: 8.62s	remaining: 5.15s
313:	learn: 0.5248425	total: 8.64s	remaining: 5.12s
314:	learn: 0.5246590	total: 8.67s	remaining: 5.09s
315:	learn: 0.5243524	total: 8.71s	remaining: 5.07s
316:	learn: 0.5242254	total: 8.74s	remaining: 5.05s
317:	learn: 0.5239616	total: 8.78s	remaining: 5.02s
318:	learn: 0.5239457	total: 8.8s	remaining: 4.99s
319:	learn: 0.5239345	total: 8.82s	remaining: 4.96s
320:	learn: 0.5238400	total: 8.86s	remaining: 4.94s
321:	learn: 0.5234327	total: 8.89s	remaining: 4.92s
322:	learn: 0.5232784	total: 8.93s	remaining: 4.89s
323:	learn: 0.5231386	total: 8.96s	remaining: 4.87s
324:	learn: 0.5229607	total: 9s	remaining: 4.84s
325:	learn: 0.5226728	total: 9.02s	remaining: 4.82s
326:	learn: 0.5225771	total: 9.06s	remaining: 4.79s
327:	learn: 0.5223513	total: 9.09s	remaining: 4.76s
328:	learn: 0.5219893	total: 9.12s	remaining: 4.74s
329:	learn: 0.5217153	total: 9.15s	remaining: 4.71s
330:	learn: 0.52

472:	learn: 0.4920691	total: 14s	remaining: 799ms
473:	learn: 0.4918339	total: 14s	remaining: 770ms
474:	learn: 0.4917096	total: 14.1s	remaining: 741ms
475:	learn: 0.4915937	total: 14.1s	remaining: 712ms
476:	learn: 0.4912896	total: 14.1s	remaining: 682ms
477:	learn: 0.4910361	total: 14.2s	remaining: 652ms
478:	learn: 0.4908584	total: 14.2s	remaining: 623ms
479:	learn: 0.4907359	total: 14.3s	remaining: 594ms
480:	learn: 0.4904215	total: 14.3s	remaining: 565ms
481:	learn: 0.4899858	total: 14.3s	remaining: 535ms
482:	learn: 0.4897964	total: 14.4s	remaining: 506ms
483:	learn: 0.4897046	total: 14.4s	remaining: 476ms
484:	learn: 0.4896904	total: 14.4s	remaining: 447ms
485:	learn: 0.4896670	total: 14.5s	remaining: 417ms
486:	learn: 0.4895132	total: 14.5s	remaining: 387ms
487:	learn: 0.4892036	total: 14.6s	remaining: 358ms
488:	learn: 0.4891736	total: 14.6s	remaining: 329ms
489:	learn: 0.4889924	total: 14.7s	remaining: 299ms
490:	learn: 0.4887030	total: 14.7s	remaining: 269ms
491:	learn: 0.48

GridSearchCV(cv=5, error_score='raise',
       estimator=<catboost.core.CatBoostClassifier object at 0x000000000A4BAEF0>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'depth': [4, 6, 8], 'learning_rate': [0.01, 0.03, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [103]:
grid.best_params_

{'depth': 6, 'learning_rate': 0.03}

In [104]:
classifier = CatBoostClassifier(iterations = 500,
                               depth = 6,
                               learning_rate = 0.03)

In [105]:
classifier.fit(X_train, y_train)

0:	learn: 0.6856227	total: 49ms	remaining: 24.4s
1:	learn: 0.6804616	total: 69.5ms	remaining: 17.3s
2:	learn: 0.6732584	total: 101ms	remaining: 16.8s
3:	learn: 0.6664787	total: 132ms	remaining: 16.3s
4:	learn: 0.6619818	total: 156ms	remaining: 15.4s
5:	learn: 0.6564947	total: 202ms	remaining: 16.6s
6:	learn: 0.6514471	total: 227ms	remaining: 16s
7:	learn: 0.6458676	total: 249ms	remaining: 15.3s
8:	learn: 0.6407333	total: 274ms	remaining: 15s
9:	learn: 0.6362380	total: 304ms	remaining: 14.9s
10:	learn: 0.6313205	total: 335ms	remaining: 14.9s
11:	learn: 0.6273909	total: 358ms	remaining: 14.6s
12:	learn: 0.6233054	total: 382ms	remaining: 14.3s
13:	learn: 0.6193368	total: 413ms	remaining: 14.4s
14:	learn: 0.6162632	total: 442ms	remaining: 14.3s
15:	learn: 0.6128671	total: 478ms	remaining: 14.4s
16:	learn: 0.6099689	total: 519ms	remaining: 14.8s
17:	learn: 0.6075819	total: 552ms	remaining: 14.8s
18:	learn: 0.6049910	total: 580ms	remaining: 14.7s
19:	learn: 0.6019045	total: 610ms	remaining: 

164:	learn: 0.5166954	total: 4.92s	remaining: 9.98s
165:	learn: 0.5165378	total: 4.94s	remaining: 9.94s
166:	learn: 0.5164843	total: 4.97s	remaining: 9.92s
167:	learn: 0.5160703	total: 5s	remaining: 9.87s
168:	learn: 0.5159527	total: 5.02s	remaining: 9.82s
169:	learn: 0.5159392	total: 5.03s	remaining: 9.77s
170:	learn: 0.5158427	total: 5.05s	remaining: 9.72s
171:	learn: 0.5156421	total: 5.07s	remaining: 9.67s
172:	learn: 0.5154027	total: 5.1s	remaining: 9.63s
173:	learn: 0.5151862	total: 5.12s	remaining: 9.6s
174:	learn: 0.5148413	total: 5.16s	remaining: 9.58s
175:	learn: 0.5146074	total: 5.18s	remaining: 9.54s
176:	learn: 0.5146063	total: 5.2s	remaining: 9.49s
177:	learn: 0.5143485	total: 5.23s	remaining: 9.46s
178:	learn: 0.5140230	total: 5.26s	remaining: 9.43s
179:	learn: 0.5137152	total: 5.28s	remaining: 9.39s
180:	learn: 0.5135731	total: 5.31s	remaining: 9.35s
181:	learn: 0.5131830	total: 5.34s	remaining: 9.32s
182:	learn: 0.5129681	total: 5.36s	remaining: 9.29s
183:	learn: 0.5127

331:	learn: 0.4859437	total: 8.93s	remaining: 4.52s
332:	learn: 0.4859424	total: 8.95s	remaining: 4.49s
333:	learn: 0.4855938	total: 8.97s	remaining: 4.46s
334:	learn: 0.4854583	total: 8.99s	remaining: 4.43s
335:	learn: 0.4853782	total: 9.02s	remaining: 4.4s
336:	learn: 0.4852320	total: 9.05s	remaining: 4.38s
337:	learn: 0.4851242	total: 9.08s	remaining: 4.35s
338:	learn: 0.4846531	total: 9.11s	remaining: 4.33s
339:	learn: 0.4846060	total: 9.15s	remaining: 4.3s
340:	learn: 0.4843318	total: 9.18s	remaining: 4.28s
341:	learn: 0.4841999	total: 9.2s	remaining: 4.25s
342:	learn: 0.4841347	total: 9.24s	remaining: 4.23s
343:	learn: 0.4838243	total: 9.26s	remaining: 4.2s
344:	learn: 0.4836967	total: 9.29s	remaining: 4.17s
345:	learn: 0.4835283	total: 9.33s	remaining: 4.15s
346:	learn: 0.4834620	total: 9.35s	remaining: 4.12s
347:	learn: 0.4833544	total: 9.37s	remaining: 4.09s
348:	learn: 0.4831135	total: 9.39s	remaining: 4.06s
349:	learn: 0.4828772	total: 9.41s	remaining: 4.03s
350:	learn: 0.48

496:	learn: 0.4635971	total: 14.1s	remaining: 84.9ms
497:	learn: 0.4633927	total: 14.1s	remaining: 56.6ms
498:	learn: 0.4633525	total: 14.1s	remaining: 28.3ms
499:	learn: 0.4633481	total: 14.1s	remaining: 0us
